Imports

In [1]:
import cv2
import numpy as np
import os

In [2]:
show_steps = False

Constants

In [3]:
# Intrinsic Matrix
K = np.array([[2564.3186869, 0, 0], [0, 2569.70273111, 0], [0, 0, 1]])

# Circle Radius
R = 10.0

Functions

In [4]:
def find_most_circular_contour(contours):

    best_circularity = 0
    best_contour = None
    best_index = -1
    
    for i, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        
        if perimeter > 0 and area > 500:  
            circularity = 4 * np.pi * area / (perimeter * perimeter)
            
            if circularity > best_circularity:
                best_circularity = circularity
                best_contour = contour
                best_index = i
    
    return best_contour, best_circularity, best_index

In [5]:
def calculate_3d_coordinates(contours, centers, camera_matrix, circle_radius):

    coordinates_3d = []
    reference_depth = None
    
    # Extract camera parameters
    fx = camera_matrix[0, 0]
    fy = camera_matrix[1, 1]
    cx = camera_matrix[0, 2]
    cy = camera_matrix[1, 2]
    
    # Find the most circular contour
    circle_contour, circularity, circle_index = find_most_circular_contour(contours)
    
    if circle_contour is not None and circularity > 0.8:

        # Calculate depth from the most circular object
        area = cv2.contourArea(circle_contour)
        radius_pixels = np.sqrt(area / np.pi)
        
        # Calculate reference depth
        reference_depth = (fx * circle_radius) / radius_pixels
            
    else:

        # Default depth
        reference_depth = 100.0 
    
    # Calculate 3D coordinates for all shapes using reference depth
    for i, (contour, center) in enumerate(zip(contours, centers)):

        # Image coordinates
        u, v = center  
        
        # Calculate 3D coordinates
        Z = reference_depth
        X = (u - cx) * Z / fx
        Y = (v - cy) * Z / fy
        coordinates_3d.append((X, Y, Z))
    
    return coordinates_3d, circle_index

In [6]:
def draw_3d_annotations(frame, contours, centers, coordinates_3d, circle_index=-1):
    
    result_frame = frame.copy()
    
    # Draw contours
    cv2.drawContours(result_frame, contours, -1, (0, 255, 0), 2)
    
    for i, (contour, center, coord_3d) in enumerate(zip(contours, centers, coordinates_3d)):
        cx, cy = center
        X, Y, Z = coord_3d
        
        # Get bounding box for text placement
        x, y, w, h = cv2.boundingRect(contour)
        
        # Draw center
        cv2.circle(result_frame, center, 5, (255, 255, 255), -1)

        # Place 3D coordinates
        text_3d = f"({X:.1f},{Y:.1f},{Z:.1f})"
        cv2.putText(result_frame, text_3d, (cx - 80, y + h + 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    return result_frame

File Names/Dirs

In [7]:
file_dir = "data"
file_name = "PennAir 2024 App Dynamic.mp4"
file = os.path.join(file_dir, file_name)

In [8]:
output_dir = os.path.join("output", "brightness")
output_name = f"annotated_3D_{file_name}"
output = os.path.join(output_dir, output_name)

Load Video

In [9]:
vid = cv2.VideoCapture(file)

In [10]:
# Video properties
fps = vid.get(cv2.CAP_PROP_FPS)
width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Video writer to save output video
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out_vid = cv2.VideoWriter(output, fourcc, fps, (width, height))

Update K

In [11]:
# Update camera matrix principal point if not provided
# (Usually cx = width/2, cy = height/2)
K_updated = K.copy()
if K_updated[0, 2] == 0:  # If cx not set
    K_updated[0, 2] = width / 2
if K_updated[1, 2] == 0:  # If cy not set
    K_updated[1, 2] = height / 2

Process Video

In [12]:
# Loop through video frames
while True:
    ret, frame = vid.read()

    # Break if no frame is returned
    if not ret:
        break
    
    # Convert to Hue Saturation Value format
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Extract only Value (Brightness)
    val = hsv[:,:,2]

    # Set brightness threshold x
    x = 140
    _, bin_val = cv2.threshold(val, x, 255, cv2.THRESH_BINARY)

    # Clean up w/ morphological operations
    smoothing_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

    # Remove noise
    no_noise = cv2.morphologyEx(bin_val, cv2.MORPH_OPEN, smoothing_kernel)

    # Fill gaps + smooth edges
    fill_gaps = cv2.morphologyEx(no_noise, cv2.MORPH_CLOSE, smoothing_kernel)

    # Smooth edges again
    smoothed = cv2.medianBlur(fill_gaps, 5)

    contours, _ = cv2.findContours(smoothed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Remove noise
    filtered_contours = []
    for contour in contours:
        area = cv2.contourArea(contour)
        # How much noise to filter
        if area > 500:
            filtered_contours.append(contour)
    
    # Calculate centers of contours
    centers = []
    for contour in filtered_contours:
        # Calculate moments
        M = cv2.moments(contour)
        if M["m00"] != 0: 
            cx = int(M["m10"] / M["m00"])  # x coord
            cy = int(M["m01"] / M["m00"])  # y coord
            centers.append((cx, cy))

    # Copy original image to draw on
    result_frame = frame.copy()

    # Draw contours
    cv2.drawContours(result_frame, filtered_contours, -1, (0, 255, 0), 2)

    # Calculate 3D coordinates and get index in list of contours of most circular object
    coordinates_3d, circle_index = calculate_3d_coordinates(filtered_contours, centers, K_updated, R)
    
    # Draw results with 3D annotations
    result_frame = draw_3d_annotations(result_frame, filtered_contours, centers, coordinates_3d, circle_index)
    
    # Write output
    out_vid.write(result_frame)
    
    if show_steps:
        # Display results
        cv2.imshow('Video Results', result_frame)
        if cv2.waitKey(1) & 0xFF == 13:
            break

vid.release()
out_vid.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

KeyboardInterrupt: 